In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, rand_score, adjusted_rand_score, mean_squared_error
from sklearn.mixture import BayesianGaussianMixture as BGM
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler

import pandas as pd
import seaborn as sn

from DataGenerator import DataGenerator
import utils

from tqdm import tqdm

In [17]:
class MLPAE(nn.Module):
  def __init__(self, in_components, out_components):
    super(MLPAE, self).__init__()

    self.in_features = in_components
    self.out_features = out_components

    self.Encoder = nn.Sequential(
      nn.Linear(in_features=self.in_features, out_features=64),
      nn.ReLU(inplace=False),
      nn.Linear(in_features=64, out_features=self.out_features),
      
      # nn.Linear(in_features=self.in_features, out_features=256),
      # nn.ReLU(inplace=False),
      # nn.Linear(in_features=256, out_features=128),
      # nn.ReLU(inplace=False),      
      # nn.Linear(in_features=128, out_features=64),
      # nn.ReLU(inplace=False),
      # nn.Linear(in_features=64, out_features=32),
      # nn.ReLU(inplace=False),  
      # nn.Linear(in_features=32, out_features=self.out_features),
      
    )

    self.Decoder = nn.Sequential(

      nn.Linear(in_features=self.out_features, out_features=64),
      # nn.Sigmoid()
      nn.ReLU(inplace=False),      
      nn.Linear(in_features=64, out_features=self.in_features),
        
      # nn.Linear(in_features=self.out_features, out_features=32),
      # nn.ReLU(inplace=False),      
      # nn.Linear(in_features=32, out_features=64),
      # nn.ReLU(inplace=False),      
      # nn.Linear(in_features=64, out_features=128),
      # nn.ReLU(inplace=False),    
      # nn.Linear(in_features=128, out_features=256),
      # nn.ReLU(inplace=False),    
      # nn.Linear(in_features=256, out_features=self.in_features),
    )

  def forward(self, x):
    encoder = self.Encoder(x)
    decoder = self.Decoder(encoder)
    return encoder, decoder

In [18]:
def train(model, device, train_loader, criterion, optimizer):
  model.train()

  train_loss = 0

  for _, (data, _) in enumerate(train_loader):
    
    data = data.to(device)
    optimizer.zero_grad()
    _, decoded_data = model(data)

    loss = criterion(data, decoded_data)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  avg_loss = train_loss / len(train_loader)
  
  return avg_loss

In [19]:
def get_datasets(model, device, train_loader, test_loader, batch_size):
  X_test = []
  y_test = []
  X_train = []
  y_train = []
    
  decode_X_train = []

  with torch.no_grad():
    for _, (data, labels) in enumerate(train_loader):
      data = data.to(device)
      labels = labels.to(device)

      transformed_data, decoded_data = model(data)
      transformed_data = np.asarray(transformed_data.detach().cpu().numpy())
      decoded_data = np.asarray(decoded_data.detach().cpu().numpy())

      X_train.append(transformed_data)
      decode_X_train.append(decoded_data)

    for _, (data, labels) in enumerate(test_loader):
      data = data.to(device)
      labels = labels.to(device)     

      transformed_data, _ = model(data)
      transformed_data = np.asarray(transformed_data.detach().cpu().numpy())

      X_test.append(transformed_data)

  X_train = np.reshape(np.asarray(X_train), (len(train_loader) * batch_size, 16))
  X_test = np.reshape(np.asarray(X_test), (len(test_loader) * batch_size, 16))
  decode_X_train = np.reshape(np.asarray(decode_X_train), (len(train_loader) * batch_size, IMAGE_DIM*IMAGE_DIM))

  return X_train, X_test, decode_X_train

In [20]:
def fit_K_means(X,n_clusters):
  kmeans = KMeans(n_clusters=n_clusters).fit(X)
  centers=kmeans.cluster_centers_
  labels=kmeans.labels_
  return centers, labels

In [21]:
#%% Try DNN on Image Dim
TOTAL_POINTS = 100000
IMAGE_DIM = 32
OUTPUT_DIM = 16
N_CLUSTERS = 10
RATIOS = np.array([0.16,0.15,0.05,0.1,0.2,0.05,0.05,0.12,0.08,0.04]) # might need random assignments for ratio
mode = 'clean' # 'clean' or 'noisy' for dae

myGenerator = DataGenerator(samples=TOTAL_POINTS, n_features=IMAGE_DIM*IMAGE_DIM, n_clusters=N_CLUSTERS, spread=20, ratio_per_cluster=RATIOS, lower_bound=0, upper_bound=100)
X, labels, centers = myGenerator.generate_data()
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.8, random_state=0)

[16000 15000  5000 10000 20000  5000  5000 12000  8000  4000]


In [22]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X = sc.transform(X)
X_test = sc.transform(X_test)

In [23]:
### extra noise
if mode == 'clean':
  X_train_dnn = X_train
elif mode == 'noisy':
  noise_factor = 0.3
  X_train_dnn = X_train + noise_factor * np.random.normal(size=X_train.shape) 
  # X_train_dnn = np.clip(X_train_dnn, 0.0, 1.0)
else:
  raise NotImplementedError(f"Data noise mode \
    {mode} has to be either 'clean' or 'noisy'.")
    
print(np.max(X_train_dnn))
print(np.min(X_train_dnn))
train_dataset = TensorDataset(torch.Tensor(X_train_dnn), torch.Tensor(y_train))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
print(X.shape)

4.5986891446746885
-4.2678662381692005
(100000, 1024)


In [24]:
#%% DNN trial
EPOCHS = 30
BATCH_SIZE = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

train_losses = np.zeros(EPOCHS)
model = MLPAE(in_components=IMAGE_DIM*IMAGE_DIM, out_components=OUTPUT_DIM).to(device)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(EPOCHS)):
  train_loss = train(model, device, train_loader, criterion, optimizer)
  # train_loss = CAE_train(model, device, train_loader, criterion, optimizer, lam=1e-4) # CAE

  train_losses[epoch] = train_loss
  print(f'Training loss for AE is: {train_loss}')

X_train_dnn, X_test_dnn, decode_X_train_dnn = get_datasets(model, device, train_loader, test_loader, batch_size=BATCH_SIZE)
print(X_train_dnn.shape)
print(X_test_dnn.shape)
print(decode_X_train_dnn.shape)

cuda


  3%|▎         | 1/30 [00:04<02:10,  4.51s/it]

Training loss for AE is: 0.11780418982729315


  7%|▋         | 2/30 [00:08<01:54,  4.10s/it]

Training loss for AE is: 0.03244680617935956


...

 97%|█████████▋| 29/30 [01:57<00:04,  4.11s/it]

Training loss for AE is: 0.03154616859741509


100%|██████████| 30/30 [02:01<00:00,  4.04s/it]

Training loss for AE is: 0.03143596573267132


(20000, 16)
(80000, 16)
(20000, 1024)


In [25]:
#fit dpgmm after pca
clf_dp_dnn = BGM(n_components=50, weight_concentration_prior_type='dirichlet_process')
clf_dp_dnn.fit(X_train_dnn)
y_pred_dp_dnn = clf_dp_dnn.predict(X_train_dnn)

a=np.unique(y_pred_dp_dnn)

centers,res_k_means=fit_K_means(X_test,len(a))

#scores
dp_arscore_dnn = adjusted_rand_score(y_test, res_k_means)
pred_clusters = set(res_k_means)

print(dp_arscore_dnn)
print(a)

1.0
[ 4  7  9 10 14 28 38 43 44 45]


In [26]:
# pca
pca = PCA(n_components=OUTPUT_DIM)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

#fit dpgmm after pca
clf_dp_pca = BGM(n_components=50, weight_concentration_prior_type='dirichlet_process')
clf_dp_pca.fit(X_train_pca)
y_pred_dp_pca = clf_dp_pca.predict(X_train_pca)

a=np.unique(y_pred_dp_pca)

centers,res_k_means=fit_K_means(X_test,len(a))

#scores
dp_arscore_pca = adjusted_rand_score(y_test, res_k_means)
pred_clusters = set(res_k_means)

print(dp_arscore_pca)
print(a)

1.0
[ 0  3  8  9 11 14 16 21 22 28]


In [ ]:
# ICA
ica = FastICA(n_components=OUTPUT_DIM)
X_train_ica = ica.fit_transform(X_train)
X_test_ica = ica.transform(X_test)

#fit dpgmm after pca
clf_dp_ica = BGM(n_components=50, weight_concentration_prior_type='dirichlet_process')
clf_dp_ica.fit(X_train_ica)
y_pred_dp_ica = clf_dp_ica.predict(X_train_ica)

a=np.unique(y_pred_dp_ica)

centers,res_k_means=fit_K_means(X_test,len(a))

#scores
dp_arscore_ica = adjusted_rand_score(y_test, res_k_means)
pred_clusters = set(res_k_means)

print(dp_arscore_ica)
print(a)

In [ ]:
# MSE
decode_X_train_pca = pca.inverse_transform(X_train_pca)
decode_X_train_ica = ica.inverse_transform(X_train_ica)

MSE_pca = mean_squared_error(decode_X_train_pca, X_train)
MSE_ica = mean_squared_error(decode_X_train_ica, X_train)
MSE_dnn = mean_squared_error(decode_X_train_dnn, X_train)
print(MSE_pca)
print(MSE_ica)
print(MSE_dnn)

In [ ]:
# Correlation matrix
df_pca = pd.DataFrame(data=X_train_pca, columns=(np.arange(X_train_pca.shape[1])))
figure = plt.figure(1, figsize=(10,8))
corr_mat_pca = df_pca.corr()
sn.heatmap(corr_mat_pca, annot=False)
plt.show()

df_dnn = pd.DataFrame(data=X_train_dnn, columns=(np.arange(X_train_dnn.shape[1])))
figure = plt.figure(2, figsize=(10,8))
corr_mat_dnn = df_dnn.corr()
sn.heatmap(corr_mat_dnn, annot=False)
plt.show()

df_ica = pd.DataFrame(data=X_train_ica, columns=(np.arange(X_train_ica.shape[1])))
figure = plt.figure(3, figsize=(10,8))
corr_mat_ica = df_ica.corr()
sn.heatmap(corr_mat_ica, annot=False)
plt.show()